In [33]:
import boto3
import re
import urllib.request as req

In [34]:
def get_public_ip():
    data = str(req.urlopen('http://checkip.dyndns.com/').read())
    
    return re.compile(r'Address: (\d+.\d+.\d+.\d+)').search(data).group(1)


def create_vpc(boto_session, cidr, name):
    ec2 = boto_session.resource('ec2')
    vpc = ec2.create_vpc(CidrBlock = cidr, 
                         TagSpecifications=[{'ResourceType': 'vpc', 'Tags': [{'Key': 'Name', 'Value': name}]}])
    vpc.wait_until_available()
    
    return vpc
    
def create_subnet(vpc, cidr, az, name):
    subnet = vpc.create_subnet(AvailabilityZone = az,
                      CidrBlock = cidr,
                      TagSpecifications=[{'ResourceType': 'subnet', 'Tags': [{'Key': 'Name','Value': name}]}])

    return subnet

def create_internet_gateway(boto_session, vpc, name):
    ec2 = boto_session.resource('ec2')
    ig = ec2.create_internet_gateway(TagSpecifications=[{'ResourceType': 'internet-gateway', 'Tags': [{'Key': 'Name','Value': name}]}])
    vpc.attach_internet_gateway(InternetGatewayId=ig.id)

    return ig

def create_route_table(vpc, name):
    rt = vpc.create_route_table(TagSpecifications=[{'ResourceType': 'route-table', 'Tags': [{'Key': 'Name','Value': name}]}])

    return rt

def create_db_subnet_group(boto_session, db_identifier, subnet_ids):
    rds = boto_session.client('rds')
    dbsubnet = rds.create_db_subnet_group(DBSubnetGroupDescription='{db} subnet group'.format(db=db_identifier),
                                          DBSubnetGroupName='{db}-subnetgroup'.format(db=db_identifier),
                                          SubnetIds=[dbsubnet01.id,dbsubnet02.id],
                                          Tags=[{'Key': 'Name','Value': '{db}-subnetgroup'.format(db=db_identifier)}])
    
    return dbsubnet

def create_basic_ec2_security_group(boto_session, vpc, name):
    ec2 = boto_session.resource('ec2')
    secgrp = ec2.create_security_group(GroupName=name,
                                   Description='basic security group for ec2',
                                   VpcId=vpc.id,
                                   TagSpecifications=[{'ResourceType': 'security-group', 'Tags': [{'Key': 'Name','Value': name}]}])
    ip22 = get_public_ip() + '/32'
    secgrp.authorize_ingress(GroupId=secgrp.id,
                             IpPermissions=[
                                 {'IpProtocol': 'tcp',
                                  'FromPort': 80,
                                  'ToPort': 80,
                                  'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                                 {'IpProtocol': 'tcp',
                                  'FromPort': 443,
                                  'ToPort': 443,
                                  'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                                 {'IpProtocol': 'tcp',
                                  'FromPort': 22,
                                  'ToPort': 22,
                                  'IpRanges': [{'CidrIp': ip22}]}
                             ])

    return secgrp

def create_mysqldb_security_group(boto_session, vpc, name):
    ec2 = boto_session.resource('ec2')
    secgrp = ec2.create_security_group(GroupName=name,
                                   Description='basic security group for mysql db',
                                   VpcId=vpc.id,
                                   TagSpecifications=[{'ResourceType': 'security-group', 'Tags': [{'Key': 'Name','Value': name}]}])
    secgrp.authorize_ingress(GroupId=secgrp.id,
                             IpPermissions=[
                                 {'IpProtocol': 'tcp',
                                  'FromPort': 3306,
                                  'ToPort': 3306,
                                  'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
                             ])
    return secgrp

In [3]:
asm = boto3.Session(profile_name = 'mjn', region_name = 'us-west-2')
projectvpc = create_vpc(asm, '213.213.48.0/24', 'tripoli-vpc')
dbsubnet01 = create_subnet(projectvpc, '213.213.48.32/27', 'us-west-2a', 'tripoli-dbsubnet-01')
dbsubnet02 = create_subnet(projectvpc, '213.213.48.64/27', 'us-west-2d', 'tripoli-dbsubnet-02')
publicsubnet = create_subnet(projectvpc, '213.213.48.96/27', 'us-west-2d', 'tripoli-publicsubnet')
dbsubnetgrp = create_db_subnet_group(asm, 'tripoli-db', [SubnetId=dbsubnet01.id, SubnetId=dbsubnet02.id])
intgateway = create_internet_gateway(asm, projectvpc, 'tripoli-ig')
routetable = create_route_table(projectvpc, 'tripoli-routetable')

In [46]:
ec2secgrp = create_basic_ec2_security_group(asm, projectvpc, 'tripoli-ec2-securitygroup')
dbsecgrp = create_mysqldb_security_group(asm, projectvpc, 'tripoli-db-securitygroup')

In [76]:
dbsubnetgrp['DBSubnetGroup']['DBSubnetGroupName']

'tripoli-db-subnetgroup'

In [ ]:
def main():
    db_identifier = 'yourDBID'
    rds = boto3.client('rds')
    try:
        rds.create_db_instance(DBInstanceIdentifier=db_identifier,
                               AllocatedStorage=20,
                               DBName='yourdbname',
                               Engine='postgres',
                               # General purpose SSD
                               StorageType='gp2',
                               StorageEncrypted=True,
                               AutoMinorVersionUpgrade=True,
                               # Set this to true later?
                               MultiAZ=False,
                               MasterUsername='youruser',
                               MasterUserPassword='yourpassword',
                               VpcSecurityGroupIds=['YOUR_SECURITY_GROUP_ID'],
                               DBInstanceClass='db.m3.2xlarge',
                               Tags=[{'Key': 'MyTag', 'Value': 'Hawaii'}])
        print 'Starting RDS instance with ID: %s' % db_identifier
    except botocore.exceptions.ClientError as e:

In [14]:
rds = asm.client('rds')

In [15]:
rds

{'DBSubnetGroup': {'DBSubnetGroupName': 'tripilo-db-subnetgroup',
  'DBSubnetGroupDescription': 'tripoli-db subnet group',
  'VpcId': 'vpc-05c7fca01add973b3',
  'SubnetGroupStatus': 'Complete',
  'Subnets': [{'SubnetIdentifier': 'subnet-0121639c0b909a328',
    'SubnetAvailabilityZone': {'Name': 'us-west-2d'},
    'SubnetOutpost': {},
    'SubnetStatus': 'Active'},
   {'SubnetIdentifier': 'subnet-02a60444a3028b638',
    'SubnetAvailabilityZone': {'Name': 'us-west-2a'},
    'SubnetOutpost': {},
    'SubnetStatus': 'Active'}],
  'DBSubnetGroupArn': 'arn:aws:rds:us-west-2:329938676177:subgrp:tripilo-db-subnetgroup'},
 'ResponseMetadata': {'RequestId': '0e6a5ae7-7c1b-4eb6-8c47-d4bd16af008c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0e6a5ae7-7c1b-4eb6-8c47-d4bd16af008c',
   'content-type': 'text/xml',
   'content-length': '1282',
   'date': 'Tue, 16 Mar 2021 00:50:01 GMT'},
  'RetryAttempts': 0}}

In [77]:
rds.create_db_instance(DBName='tripoliwordpress',
                       DBInstanceIdentifier='tripoli-db',
                       AllocatedStorage=20,
                       DBInstanceClass='db.t2.micro',
                       Engine='mysql',
                       MasterUsername='tripoliadmin',
                       MasterUserPassword='Tripoli2021!',
                       StorageType='gp2',
                       StorageEncrypted=False,
                       AutoMinorVersionUpgrade=True,
                       MultiAZ=False,
                       PubliclyAccessible=False,
                       VpcSecurityGroupIds=[dbsecgrp.id],
                       DBSubnetGroupName=dbsubnetgrp['DBSubnetGroup']['DBSubnetGroupName'],
                       Tags=[{'Key': 'Name', 'Value': 'tripoli-db'}])

{'DBInstance': {'DBInstanceIdentifier': 'tripoli-db',
  'DBInstanceClass': 'db.t2.micro',
  'Engine': 'mysql',
  'DBInstanceStatus': 'creating',
  'MasterUsername': 'tripoliadmin',
  'DBName': 'tripoliwordpress',
  'AllocatedStorage': 20,
  'PreferredBackupWindow': '13:25-13:55',
  'BackupRetentionPeriod': 1,
  'DBSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0bfb6cc71db10355f',
    'Status': 'active'}],
  'DBParameterGroups': [{'DBParameterGroupName': 'default.mysql8.0',
    'ParameterApplyStatus': 'in-sync'}],
  'DBSubnetGroup': {'DBSubnetGroupName': 'tripoli-db-subnetgroup',
   'DBSubnetGroupDescription': 'tripoli-db subnet group',
   'VpcId': 'vpc-05c7fca01add973b3',
   'SubnetGroupStatus': 'Complete',
   'Subnets': [{'SubnetIdentifier': 'subnet-0121639c0b909a328',
     'SubnetAvailabilityZone': {'Name': 'us-west-2d'},
     'SubnetOutpost': {},
     'SubnetStatus': 'Active'},
    {'SubnetIdentifier': 'subnet-02a60444a3028b638',
     'SubnetAvailabilityZone